# Building a Chatbot Homework
## Emaan Qillawala
## DS 2002 4/10/24

In [1]:
# import libraries

# Tensorflow is the program used to build the Chatbot
#!pip install tensorflow

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np

# Tip! Use TPU instead of CPU for a faster runtime

### 1. Build a chatbot using Python/TF

In [3]:
# develop questions that Chatbot can understand

questions = [
    'Hello!',
    'What is your mood for today?',
    'Can you help me with something?',
    'What is your favorite animal?',
    'How do I make cereal?',
    'Do you know the movie Wall-E?',
    'Who won the NCAA Mens basketball tournament in 2019?',
    'Where is Morocco located?',
    'Where can I find the longest river in the United States?',
    'What are the three primary colors?',
    'Are you draining my computer battery?',
    'What is acceleration due to gravity?',
    'Can you talk to like a pirate?',
    'Goodbye!'
]

In [4]:
# develop answers to Chatbot questions

answers = [
    'Hello there! I am Chatbot.',
    'I am feeling good today! :)',
    'Sure! What can I help you with?',
    'Of course, my favorite animal is the robot.',
    'You pour cereal into a bowl FIRST and then add milk. Enjoy!',
    'I am a chatbot. What do you think?',
    'UVA, wahoowa!',
    'Morocco is a country located in Africa.',
    'You can find the Mississippi River in many states, including Mississippi.',
    'The three primary colors are Red, Blue, and Yellow.',
    'That is not particularly due to me.',
    'Acceleration due to gravity is approximately 9.8 m/s^2.',
    'Arggg! Yes I can me hearty!',
    'Goodbye user. See you next time!'
]

### 2. Customize and train Chatbot responses

In [5]:
# tokenize the sentences for Chatbot to understand

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

In [6]:
# pad the sequences -> giving input sequences a specified length

max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

# define the vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [7]:
# define model parameters

embedding_dim = 256
units = 1024

In [8]:
# define the encoder model to generate Chatbot outputs

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

In [9]:
# define the decoder model to generate Chatbot outputs

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [10]:
# define the seq2seq model using the encoder and decoder models

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# summary of the model -> shows all of the components of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            26624     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            26624     ['input_2[0][0]']             
                                                                                              

### 3. Practice basic inputs/outputs
### -> ex: hello and goodbye

In [11]:
# prepare decoder input data that contains the start token only

decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1  # 1 is the start token

In [12]:
# train the model with the decoder input data and define number of epochs

model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)

Epoch 1/100
7/7 [==============================] - 12s 1s/step - loss: 4.2014 - accuracy: 0.3393
Epoch 2/100
7/7 [==============================] - 7s 1s/step - loss: 3.4441 - accuracy: 0.3810
Epoch 3/100
7/7 [==============================] - 7s 1s/step - loss: 3.3549 - accuracy: 0.3929
Epoch 4/100
7/7 [==============================] - 8s 1s/step - loss: 3.0754 - accuracy: 0.3869
Epoch 5/100
7/7 [==============================] - 7s 1s/step - loss: 3.0927 - accuracy: 0.3929
Epoch 6/100
7/7 [==============================] - 8s 1s/step - loss: 3.0077 - accuracy: 0.3929
Epoch 7/100
7/7 [==============================] - 7s 996ms/step - loss: 3.1436 - accuracy: 0.3690
Epoch 8/100
7/7 [==============================] - 8s 1s/step - loss: 2.8819 - accuracy: 0.3988
Epoch 9/100
7/7 [==============================] - 7s 986ms/step - loss: 2.8587 - accuracy: 0.3810
Epoch 10/100
7/7 [==============================] - 8s 1s/step - loss: 2.8322 - accuracy: 0.3988
Epoch 11/100
7/7 [==============

In [25]:
# implement the Chatbot function to talk to Chatbot
## process the inputs
## produce the outputs

def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # Start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # End token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

In [26]:
# Chat with Chatbot!

input_text = "Can you help me with something?"
print(f"You: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")

You: Can you help me with something?
1/1 [==============================] - 0s 109ms/step
Bot: sure what can i help with think


In [27]:
# create an interactive box to talk to Chatbot

print("Start chatting with the bot! Type 'done' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'done':
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")

# include an option to end the chat by typing 'done' into the box

Start chatting with the bot! Type 'done' to exit.
You: Hello@
1/1 [==============================] - 0s 69ms/step
Bot: goodbye user see next time
You: Hello!
1/1 [==============================] - 0s 70ms/step
Bot: goodbye user see next time
You: How do I make cereal?
1/1 [==============================] - 0s 74ms/step
Bot: pour cereal into a bowl first and then add milk
You: done
